In [1]:
import gcs_utils
import pull
import os
from google.cloud import documentai
from google.api_core.client_options import ClientOptions

bucket_name = 'document-ai-test-jd'
project_id = 'car-price-prediction-400719'
location = 'eu'
processor_id = '5b8635a23997916a'


# for pdf in gcs_utils.list_blobs(bucket_name):
#     if pdf.endswith(".pdf"):
#         print(f'Downloading {pdf}')
#         pull.download_blob(bucket_name, pdf, pdf)



def process_document(
        project_id=project_id,
        location=location,
        processor_id=processor_id, 
        file=""):
    # Instantiates a client
    client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor version, e.g.:
    # `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    # You must create a processor before running this sample.
    name = client.processor_version_path(
        project_id, location, processor_id, 'pretrained-invoice-v1.5-2023-09-15'
    )

    # Read the file into memory
    with open(file, "rb") as image:
        image_content = image.read()

    # Configure the process request
    request = documentai.ProcessRequest(
        name=name,
        raw_document=documentai.RawDocument(content=image_content, mime_type="application/pdf"),
        # Only supported for Document OCR processor
    )

    result = client.process_document(request=request)

    # For a full list of `Document` object attributes, reference this page:
    # https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
    return result.document

for pdf in os.listdir():
    if pdf.endswith(".pdf"):

        document=process_document(project_id=project_id,
                                  location=location,
                                  processor_id=processor_id, 
                                  file=pdf)
        print(document)

uri: ""
mime_type: "application/pdf"
text: "Sliced Invoices\nInvoice\nInvoice Number\nINV-3337\nFrom:\nDEMO - Sliced Invoices\nOrder Number\n12345\nSuite 5A-1204\nInvoice Date\nJanuary 25, 2016\n123 Somewhere Street\nDue Date\nJanuary 31, 2016\nYour City AZ 12345\nadmin@slicedinvoices.com\nTotal Due\n$93.50\nTo:\nTest Business\n123 Somewhere St\nMelbourne, VIC 3000\ntest@test.com\nHrs/Qty\nService\nRate/Price\nAdjust\nSub Total\n1.00\nWeb Design\nThis is a sample description...\n$85.00\n0.00%\n$85.00\nSub Total\n$85.00\nTax\n$8.50\nTotal\n$93.50\nANZ Bank\nPax\nACC # 1234 1234\nBSB #4321 432\nPayment is due within 30 days from date of invoice. Late payment is subject to fees of 5% per month.\nThanks for choosing DEMO - Sliced Invoices I admin@slicedinvoices.com\nPage 1/1\n"
pages {
  page_number: 1
  image {
    content: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\006\221\000\000\tJ\010\002\000\000\000In\256\010\000\000 \000IDATx\234\354\335\347s\034y\236&\366\364UY\336\301{\002 \000\

In [33]:
dict_row = []
for entity in document.entities:
    dictionary = {}
    dictionary["Entity_type"] =  entity.type_
    dictionary["mentioned_text"] =  entity.mention_text
    dictionary["parsed_text"] =  entity.normalized_value.text
    dictionary["confidence"] =  entity.confidence * 100
    dict_row.append(dictionary)

In [37]:
import pandas as pd
pd.DataFrame(dict_row)

,Entity_type,mentioned_text,parsed_text,confidence
0,due_date,"January 31, 2016",2016-01-31,99.304628
1,invoice_id,INV-3337,,98.971218
2,invoice_date,"January 25, 2016",2016-01-25,98.792720
3,total_amount,93.50,93.5,97.687280
4,total_tax_amount,8.50,8.5,97.700620
5,net_amount,85.00,85,96.059591
6,currency,$,USD,75.659221
7,payment_terms,Payment is due within 30 days from date of inv...,,70.384735
8,invoice_type,,invoice_statement,59.924787
9,supplier_address,Suite 5A-1204\n123 Somewhere Street\nYour City...,,56.465530
